___
# **TELEBOT EQUIPO VERDE OSCURO**
___

# **Bot de las ciudades españolas: Una Herramienta Interactiva para Consultas Climáticas y Geográficas**

El **Bot de las ciudades Españolas** es una solución interactiva diseñada para proporcionar información climática y geográfica detallada y personalizada sobre diversas ciudades, basada en **medias mensuales históricas de los últimos 20 años**. A través de un flujo conversacional estructurado, el bot permite al usuario obtener gráficos climáticos, reportes PDF, y datos adicionales sobre las ciudades seleccionadas. La interacción con el bot está optimizada para facilitar la navegación y garantizar precisión en las respuestas.

### **Fuente de Datos: Datos Climáticos Históricos**
El bot utiliza un **dataset climáticos en formato CSV** ('clima_ciudades_dataset.csv'), generados a partir de promedios históricos proporcionados por fuente confiables como:
- **AEMET** (Agencia Estatal de Meteorología de España),
Estos datos se ajustaron considerando medias históricas para cada ciudad y mes, asegurando representaciones precisas y fiables.

---

### **Funciones y Características**
#### 1. **Inicio e Interacción Personalizada**
La interacción con el bot comienza a través de los comandos **/start** o **/help**, que:
- Dan la bienvenida al usuario.
- Solicitan el **nombre** del usuario para personalizar las respuestas y mejorar la experiencia interactiva.  
Este enfoque refuerza la usabilidad y hace que la interacción sea más fluida.

#### 2. **Consulta Climática Detallada**
El comando **/Ciudad** permite al usuario realizar consultas climáticas específicas. El proceso se desarrolla en los siguientes pasos:
1. **Petición del nombre de la ciudad:** El usuario ingresa el nombre de la ciudad deseada.
2. **Selección del mes:** El bot solicita el mes en el cual se desea obtener información climática.  
3. **Generación de gráficos y análisis:** Se presentan gráficos interactivos con datos comparativos como:
   - Temperatura media.
   - Precipitación.
   - Humedad relativa, entre otros.
4. **Generación de reportes en PDF:** El bot crea un informe detallado en formato PDF que incluye:
   - Gráficos estadísticos.
   - Resúmenes climáticos basados en análisis estadísticos descriptivos.  

#### 3. **Información Adicional de la Ciudad**
Tras obtener los resultados climáticos, el bot pregunta al usuario si desea conocer más sobre la ciudad seleccionada. En caso afirmativo:
- Utiliza la **API de Wikipedia** (a través de `wikipediaapi`) para extraer información relevante y mostrarla directamente en pantalla.  
Esta funcionalidad incluye datos como historia, cultura, lugares de interés, entre otros.

#### 4. **Consulta Geográfica con GeoDB**
El comando **/Ciudad_info** amplía las capacidades del bot al proporcionar información geográfica detallada sobre ciudades españolas.  
A través de la **API de GeoDB**, el bot obtiene:
- Coordenadas geográficas.
- Tamaño de la población.
- Altitud sobre el nivel del mar.
- Otros datos útiles para planificar visitas o realizar análisis geográficos.  

#### 5. **Visualización de Mapas con Mapbox**
Usando las coordenadas obtenidas en **/Ciudad_info**, el bot permite al usuario generar un mapa interactivo de la ciudad mediante el comando **/Mapa**.  
Esta funcionalidad aprovecha la **API de Mapbox**, que genera:
- Mapas estáticos de alta calidad.
- La ubicación específica marcada con un pin, basada en las coordenadas ingresadas por el usuario.  
El mapa se envía como una imagen al usuario, acompañada de un mensaje descriptivo.

---

### **Flujo del Proceso**
1. **Inicio del flujo:** El usuario ejecuta `/start` o `/help` para comenzar la interacción.
2. **Personalización:** El bot solicita el nombre del usuario.
3. **Consulta climática:**
   - El usuario proporciona la ciudad y el mes deseados.
   - El bot genera gráficos y estadísticas climáticas.
   - Se genera y envía un informe PDF.
4. **Información adicional de la ciudad:** El bot consulta la API de Wikipedia para mostrar detalles sobre la ciudad seleccionada.
5. **Consulta geográfica (opcional):** Con **/Ciudad_info**, el usuario puede obtener datos geográficos detallados mediante la API de GeoDB.
6. **Visualización del mapa:** El usuario puede visualizar mapas estáticos utilizando el comando **/Mapa**, introduciendo las coordenadas extraídas previamente.
7. **Reinicio o despedida:** El bot cierra la interacción y ofrece opciones para realizar nuevas consultas.

---

### **Optimización y Control de Errores**
Se han implementado varias medidas para garantizar la robustez y la experiencia de usuario:
1. **Validación de entrada del usuario:**  
   - El bot valida los datos ingresados, como coordenadas y nombres de ciudades, para evitar errores de procesamiento.  
   - Si el usuario introduce coordenadas inválidas en `/Mapa`, el bot responde con instrucciones claras sobre el formato correcto.  
   
2. **Flujo guiado:**  
   - En cada paso, el bot proporciona instrucciones detalladas, minimizando la posibilidad de confusión del usuario.  
   - Cada comando tiene mensajes de ayuda incorporados.  

3. **Manejo de errores de API:**  
   - En caso de que una API falle o no pueda proporcionar resultados, el bot informa al usuario del problema y evita interrupciones abruptas.  

4. **Prevención de uso sin registro:**  
   - El bot verifica si el usuario está registrado antes de permitir el uso de ciertas funciones (como `/Mapa`), y en caso de no estar registrado, solicita que complete el registro con el comando **/Registro**.  

---

### **Tecnologías y APIs Utilizadas**
El bot integra varias tecnologías modernas para ofrecer una experiencia completa:
1. **API de Wikipedia:**  
   Para extraer información descriptiva y cultural sobre las ciudades seleccionadas.
2. **API de GeoDB:**  
   Para consultar datos geográficos de ciudades españolas.  
3. **API de Mapbox:**  
   Para generar mapas estáticos con marcadores específicos.  
4. **Librerías de Python:**  
   - `matplotlib`: Generación de gráficos interactivos.  
   - `pdfkit` o `fpdf`: Creación de reportes PDF personalizados.  
   - `pandas`: Procesamiento y análisis de datos climáticos en CSV.  
   - `requests`: Integración con APIs externas.  

---

### **Ventajas del Bot del Clima**
- **Experiencia personalizada:** Gracias a la personalización basada en el nombre del usuario.  
- **Interacción fluida:** El flujo guiado asegura que los usuarios puedan realizar consultas fácilmente.  
- **Resultados precisos:** Los datos históricos y las APIs garantizan información relevante y confiable.  
- **Multifuncionalidad:** Desde estadísticas climáticas hasta mapas y datos geográficos, el bot cubre diversas necesidades.  

---

El **Bot del Clima** es una herramienta versátil y eficiente que integra análisis de datos, APIs geográficas y generación de informes, proporcionando al usuario información completa y fácilmente accesible. Ideal para viajeros, analistas o cualquier persona interesada en el clima y las características de una ciudad.


___
### **1.- Librerías y preparación del Telebot**
___

In [72]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import os
import telebot
import re
from telebot import types
import pydub
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from telebot.types import ReplyKeyboardMarkup, KeyboardButton, ReplyKeyboardRemove
import warnings
import wikipediaapi
import requests
from io import BytesIO
warnings.filterwarnings(action='ignore')

In [73]:
# Ruta del archivo CSV con los datos climáticos
csv_path = "clima_ciudades_dataset.csv"

# Crear el bot de Telegram utilizando el token de la API proporcionado por telegram
API_TOKEN = "7761290773:AAH-2mKnpiRp8GfCAxLp1wZWV7CGiNnyr-8"
bot = telebot.TeleBot(API_TOKEN)

# Memoria del chatbot, es un diccionario que almacena información del usuario durante la conversación
chatbot_memory = {}

___
### **2.- Funciones**
___

In [74]:
# Función para procesar el archivo de audio (de formato OGG a texto)
def procesar_audio(audio_file):
    try:
        # Usamos pydub para convertir el archivo de audio OGG a formato WAV
        audio = pydub.AudioSegment.from_file(audio_file, format="ogg")
        wav_file = "voice.wav"
        audio.export(wav_file, format="wav")

        # Utilizamos la librería SpeechRecognition para convertir el archivo WAV en texto
        import speech_recognition as sr
        recognizer = sr.Recognizer()
        
        # Abrimos el archivo WAV y lo procesamos para obtener el texto
        with sr.AudioFile(wav_file) as source:
            audio_data = recognizer.record(source)
            texto = recognizer.recognize_google(audio_data, language="es-ES")
            return texto
    except Exception as e:
        # Si ocurre un error durante el proceso, lo capturamos y devolvemos un mensaje de error
        return f"Error al procesar el audio: {e}"


In [75]:
def generar_pdf(chat_id, ciudad, mes):
    # Definimos el nombre del archivo PDF
    pdf_filename = f"resumen_climatico_{ciudad}_{mes}.pdf"
    
    # Creamos un objeto canvas para el PDF
    c = canvas.Canvas(pdf_filename, pagesize=letter)
    
    # Escribimos la información climática en el PDF
    df = pd.read_csv(csv_path)
    datos_ciudad = df[(df["Ciudad"] == ciudad) & (df["Mes"] == mes)].iloc[0]
    
    c.setFont("Helvetica", 12)
    c.drawString(30, 750, f"Resumen Climático de {ciudad} en {mes}:")
    c.drawString(30, 730, f"• Temperatura Media: {datos_ciudad['Temperatura_Media']}°C")
    c.drawString(30, 710, f"• Precipitación: {datos_ciudad['Precipitacion_mm']} mm")
    c.drawString(30, 690, f"• Horas de Sol: {datos_ciudad['Horas_Sol']} horas")
    c.drawString(30, 670, f"• Humedad Relativa: {datos_ciudad['Humedad_Relativa']}%")
    
    # Agregamos los gráficos generados al PDF
    c.drawImage("grafico_comparativo.png", 30, 200, width=550, height=400)  # Ajusta el tamaño de la imagen
    
    # Guardamos el archivo PDF
    c.save()

    # Enviamos el PDF al usuario
    with open(pdf_filename, "rb") as f:
        bot.send_document(chat_id, f)
    
    # Eliminamos el archivo PDF después de enviarlo
    os.remove(pdf_filename)

In [76]:
# Función para generar los gráficos comparativos (barras, líneas y un nuevo gráfico en lugar de la correlación)
def generar_grafico_comparativo(chat_id, ciudad, mes):
    df = pd.read_csv(csv_path)
    datos_ciudad = df[(df["Ciudad"] == ciudad) & (df["Mes"] == mes)]  # Filtramos los datos para la ciudad y mes seleccionados
    datos_mes = df[df["Mes"] == mes]  # Filtramos todos los datos para el mes seleccionado

    if datos_ciudad.empty:
        bot.send_message(chat_id, f"Lo siento, no tengo datos para {ciudad} en {mes}.")
        return

    # Creamos una figura grande para los gráficos
    plt.figure(figsize=(14, 10))

    # Gráfico de barras para la comparación de temperaturas
    plt.subplot(2, 2, 1)
    sns.barplot(data=datos_mes, x="Ciudad", y="Temperatura_Media", palette="Blues", ci=None)
    plt.title(f"Comparación de Temperaturas - {mes}")
    plt.xticks(rotation=45)
    
    # Resaltar la ciudad seleccionada en el gráfico de barras de temperaturas (en verde)
    for index, city in enumerate(datos_mes["Ciudad"].unique()):
        if city == ciudad:
            plt.gca().patches[index].set_color('green')  # Resaltamos en verde la ciudad seleccionada

    # Gráfico de barras para precipitaciones
    plt.subplot(2, 2, 2)
    sns.barplot(data=datos_mes, x="Ciudad", y="Precipitacion_mm", palette="Blues", ci=None)
    plt.title(f"Comparación de Precipitaciones - {mes}")
    plt.xticks(rotation=45)

    # Resaltar la ciudad seleccionada en el gráfico de barras de precipitaciones (en verde)
    for index, city in enumerate(datos_mes["Ciudad"].unique()):
        if city == ciudad:
            plt.gca().patches[index].set_color('green')  # Resaltamos en verde la ciudad seleccionada

    # Gráfico de evolución climática para la ciudad seleccionada
    datos_ciudad_completa = df[df["Ciudad"] == ciudad]
    plt.subplot(2, 2, 3)
    sns.lineplot(data=datos_ciudad_completa, x="Mes", y="Temperatura_Media", label="Temperatura Media", marker="o")
    sns.lineplot(data=datos_ciudad_completa, x="Mes", y="Horas_Sol", label="Horas de Sol", marker="o")
    plt.title(f"Evolución Climática en {ciudad}")
    plt.legend()
    plt.xticks(rotation=45)

    # Nuevo gráfico: Evolución de la Temperatura Media durante el mes
    plt.subplot(2, 2, 4)
    sns.lineplot(data=datos_ciudad_completa, x="Mes", y="Temperatura_Media", label="Temperatura Media", marker="o", color="orange")
    plt.title("Evolución de la Temperatura Media en el mes")
    plt.xticks(rotation=45)

    # Ajustamos el diseño de los gráficos y agregamos el título general
    plt.tight_layout()
    plt.suptitle(f"Análisis Climático de {ciudad} y Comparación en {mes}", y=1.02)
    plt.savefig("grafico_comparativo.png")  # Guardamos la imagen generada
    plt.close()  # Cerramos la figura

    # Enviamos el gráfico al usuario
    with open("grafico_comparativo.png", 'rb') as photo:
        bot.send_photo(chat_id, photo)
    generar_pdf(chat_id, ciudad, mes)



In [77]:
# Función para enviar el resumen de las métricas climáticas de la ciudad seleccionada
def resumen_climatico(chat_id, ciudad, mes):
    df = pd.read_csv(csv_path)
    datos_ciudad = df[(df["Ciudad"] == ciudad) & (df["Mes"] == mes)].iloc[0]  # Obtenemos los datos de la ciudad y mes

    # Creamos el mensaje con las métricas climáticas de la ciudad
    mensaje = (f"*Resumen climático de {ciudad} en {mes}:*\n\n"
               f"• **Temperatura Media**: {datos_ciudad['Temperatura_Media']}°C 🌡️\n"
               f"• **Precipitación**: {datos_ciudad['Precipitacion_mm']} mm 🌧️\n"
               f"• **Horas de Sol**: {datos_ciudad['Horas_Sol']} horas 🌞\n"
               f"• **Humedad Relativa**: {datos_ciudad['Humedad_Relativa']}% 💧\n")

    # Enviamos el resumen al usuario en formato Markdown
    bot.send_message(chat_id, mensaje, parse_mode='Markdown')


In [78]:
# Definir un tamaño máximo de caracteres por mensaje 
MAX_MESSAGE_LENGTH = 1000 #(para que no se alargue mucho)

def enviar_mensajes_largos(chat_id, mensaje):
    """Envía el mensaje en partes si es más largo que el límite permitido,
    y trata de no cortar las oraciones por la mitad."""
    
    while len(mensaje) > MAX_MESSAGE_LENGTH:
        # Buscamos la última posición del punto dentro del límite
        limite = mensaje.rfind('.', 0, MAX_MESSAGE_LENGTH)  # Buscar punto dentro de los primeros 2000 caracteres

        if limite == -1:
            # Si no encontramos un punto, cortamos al final del límite
            limite = MAX_MESSAGE_LENGTH

        # Enviamos la parte del mensaje y la recortamos para el siguiente envío
        bot.send_message(chat_id, mensaje[:limite+1])
        mensaje = mensaje[limite+1:].strip()  # Recortamos la parte ya enviada

    # Enviamos el resto del mensaje (si hay)
    if mensaje:
        bot.send_message(chat_id, mensaje)

    # Ahora que hemos enviado el mensaje, verificamos si se ha alcanzado el límite máximo y no se envían más mensajes.
    # Aquí es donde puedes detener la ejecución si lo deseas.
    if len(mensaje) > MAX_MESSAGE_LENGTH:
        return  # Aquí terminamos sin enviar más partes


In [79]:
def obtener_info_extra(ciudad, chat_id):
    # Inicializamos Wikipedia con un user agent válido
    wiki_wiki = wikipediaapi.Wikipedia(
        language='es', 
        user_agent="MiBotClima/1.0 (miemail@dominio.com)"
    )
    
    # Buscamos la página de la ciudad
    pagina = wiki_wiki.page(ciudad)

    # Verificamos si la página existe
    if pagina.exists():
        # Extraemos el resumen y eliminamos las referencias
        resumen = pagina.summary
        resumen_limpio = re.sub(r'\[\d+\]', '', resumen)  # Elimina las referencias

        # Enviamos el resumen limpio de la ciudad (dividido si es necesario)
        enviar_mensajes_largos(chat_id, resumen_limpio)

        
    else:
        # Si no existe, informamos al usuario
        bot.send_message(chat_id, f"No pude encontrar información adicional sobre {ciudad} en Wikipedia.")

In [80]:
# Función para procesar la respuesta sobre más información
def process_more_info_response(message):
    respuesta = message.text.lower()
    chat_id = message.chat.id

    if respuesta in ['sí', 'si']:
        ciudad = chatbot_memory[chat_id]['ciudad']  # Obtenemos la ciudad guardada en la memoria
        obtener_info_extra(ciudad, chat_id)  # Llamamos a la función para obtener más información sobre la ciudad
        # Volvemos a preguntar después de mostrar la información extra
        ask_for_more_info(message)
    elif respuesta == 'no':
    # Mensaje final con botones de acción
        final_message = (
        """🎉 Gracias por consultar la información de la ciudad. 
        
        - Utiliza /start o /help para iniciar otra vez"""
    )
    

        bot.send_message(chat_id, final_message, reply_markup=ReplyKeyboardRemove())

        chatbot_memory.pop(chat_id, None)  # Eliminamos la memoria del chat para limpiar los datos
    else:
        bot.send_message(chat_id, "Por favor, responde 'Sí' o 'No'.")
        ask_for_more_info(message)  # Repetimos la pregunta si la respuesta no es válida

# Función para preguntar si desea más información sobre la ciudad
def ask_for_more_info(message):
    chat_id = message.chat.id

    # Creamos los botones de Sí y No
    markup = ReplyKeyboardMarkup(one_time_keyboard=True, resize_keyboard=True)
    markup.add(KeyboardButton('Sí'), KeyboardButton('No'))

    # Enviamos el mensaje preguntando si desea más información
    bot.send_message(chat_id, "¿Te gustaría obtener más información sobre la ciudad? (Puedes escribir 'Sí' o 'No').", reply_markup=markup)

    # Registramos el siguiente paso para procesar la respuesta
    bot.register_next_step_handler_by_chat_id(chat_id, process_more_info_response)


___
### **3.- Puesta en marcha**
___

In [ ]:
geodb_api_key = "0e14a85e85msh12317d21afa936fp113c8djsn75b6a1a6473f" #API GEODB
geodb_base_url = "https://wft-geo-db.p.rapidapi.com/v1/geo/cities"

usuario = None

# Función que maneja el comando /start o /help
@bot.message_handler(commands=['start', 'help'])
def first_step(message):
    chat_id = message.chat.id
    chatbot_memory[chat_id] = {}  # Inicializamos la memoria del chat para el usuario

    # Enviar mensaje inicial con instrucciones
    bot.send_message(chat_id, 
                 "¡Hola! 👋 Bienvenido al bot de las ciudades Españolas 🇪🇸.\n\n"
                 "Por favor, primero, regístrate con /Registro 📝. \n\n"
                 "Luego, usa /Ciudad 🌍 para seleccionar una ciudad y un mes y obtener un análisis climático ☀️🌧️. \n\n"
                 "Con /Ciudad_info 🗺️ podrás obtener información geográfica de la ciudad que quieras mediante la API de GeoDB 🌐. \n\n"
                 "Usando /Mapa [Latitud, Longitud]: Podrás visualizar el mapa de la ciudad Española deseada 🗺️")
    
# Función para procesar el nombre del usuario
@bot.message_handler(commands=['Registro'])
def second_step(message):
    chat_id = message.chat.id

    # Si el nombre es un texto, lo guardamos directamente, si es voz, procesamos el archivo
    if message.text.lower() == "/registro":
        bot.send_message(chat_id, "Por favor, escribe tu nombre para completar el registro.")
        bot.register_next_step_handler(message, save_name)
        return

# Función para guardar el nombre del usuario
def save_name(message):
    chat_id = message.chat.id
    nombre = message.text  # Ahora se guarda correctamente el nombre
    global usuario 
    usuario = nombre
    chatbot_memory[chat_id]['nombre'] = nombre  # Guardamos el nombre en la memoria del chat

    # Mensaje confirmatorio y solicitud para continuar con /Ciudad
    bot.send_message(chat_id, f"Encantado, {usuario}! Ahora, para continuar, escribe el comando /Ciudad para seleccionar una ciudad y un mes para el análisis climático, o /Ciudad_info. \n\nCuando obtengas coordenadas con Ciudad_info, puedes visualizar el mapa de la ciudad usando el comando /Mapa")

# Función para guardar el nombre del usuario

def procesar_nombre(message):
    chat_id = message.chat.id
    nombre = message.text.strip()

    # Guardar el nombre en la memoria del chatbot
    if chat_id not in chatbot_memory:
        chatbot_memory[chat_id] = {}

    chatbot_memory[chat_id]['nombre'] = nombre

    bot.send_message(chat_id, f"Encantado, {nombre}! Ahora, para continuar, escribe el comando /Ciudad para seleccionar una ciudad y un mes para el análisis climático, o /Ciudad_info.")

# Función que maneja el comando /Ciudad
@bot.message_handler(commands=['Ciudad'])
def third_step(message):
    chat_id = message.chat.id

    # if 'nombre' not in chatbot_memory[chat_id]:  # Comprobamos si el usuario se registró
    #     bot.send_message(chat_id, "⚠️ Debes registrarte antes de usar este comando. Usa /Registro para registrarte. 📝")
    #     return
    if 'nombre' not in chatbot_memory.get(chat_id, {}):  # Verificamos si el usuario se registró
        bot.send_message(chat_id, "⚠️ Debes registrarte antes de usar este comando. Usa /Registro para registrarte. 📝")
        return
    
    # Mostrar lista de ciudades disponibles
    df = pd.read_csv(csv_path)
    ciudades = df["Ciudad"].unique()

    markup = types.ReplyKeyboardMarkup(one_time_keyboard=True, resize_keyboard=True)
    for ciudad in ciudades:
        markup.add(types.KeyboardButton(ciudad))

    ultimo_mensaje = bot.send_message(chat_id, "Por favor, selecciona una ciudad española que quieras visitar:", reply_markup=markup)
    bot.register_next_step_handler(ultimo_mensaje, select_month)

# Función para procesar la ciudad seleccionada
def select_month(message):
    chat_id = message.chat.id
    ciudad = message.text.capitalize()

    df = pd.read_csv(csv_path)
    ciudades = df["Ciudad"].unique()

    if ciudad in ciudades:
        chatbot_memory[chat_id]['ciudad'] = ciudad

        # Mostrar lista de meses disponibles
        meses = df["Mes"].unique()
        markup = types.ReplyKeyboardMarkup(one_time_keyboard=True, resize_keyboard=True)
        for mes in meses:
            markup.add(types.KeyboardButton(mes))

        ultimo_mensaje = bot.send_message(chat_id, f"Perfecto, {usuario}. Ahora dime el mes que te interesa:", reply_markup=markup)
        bot.register_next_step_handler(ultimo_mensaje, process_month)
    else:
        bot.send_message(chat_id, "No he encontrado esa ciudad. Por favor, selecciona una de la lista.")
        ask_for_city(message)

# Función para pedir ciudad nuevamente si no está en la lista
def ask_for_city(message):
    chat_id = message.chat.id
    df = pd.read_csv(csv_path)
    ciudades = df["Ciudad"].unique()

    markup = types.ReplyKeyboardMarkup(one_time_keyboard=True, resize_keyboard=True)
    for ciudad in ciudades:
        markup.add(types.KeyboardButton(ciudad))

    ultimo_mensaje = bot.send_message(chat_id, "Por favor, selecciona o escribe una ciudad española que quieras visitar:", reply_markup=markup)
    bot.register_next_step_handler(ultimo_mensaje, select_month)

# Función para procesar el mes seleccionado
def process_month(message):
    chat_id = message.chat.id
    mes = message.text.capitalize()

    df = pd.read_csv(csv_path)
    meses = df["Mes"].unique()

    if mes in meses:
        chatbot_memory[chat_id]['mes'] = mes
        ciudad = chatbot_memory[chat_id]['ciudad']

        # Llamada a funciones para generar gráficos y resúmenes climáticos
        generar_grafico_comparativo(chat_id, ciudad, mes)
        resumen_climatico(chat_id, ciudad, mes)

        ask_for_more_info(message)

    else:
        bot.send_message(chat_id, "No he encontrado ese mes. Por favor, selecciona uno de la lista.")
        ask_for_month(message)

# Función para pedir mes nuevamente
def ask_for_month(message):
    chat_id = message.chat.id
    df = pd.read_csv(csv_path)
    meses = df["Mes"].unique()

    markup = types.ReplyKeyboardMarkup(one_time_keyboard=True, resize_keyboard=True)
    for mes in meses:
        markup.add(types.KeyboardButton(mes))

    ultimo_mensaje = bot.send_message(chat_id, "Por favor, selecciona o escribe un mes válido de la lista:", reply_markup=markup)
    bot.register_next_step_handler(ultimo_mensaje, process_month)

# Función para obtener información adicional de una ciudad usando GeoDB
@bot.message_handler(commands=['Ciudad_info'])
def city_info(message):
    chat_id = message.chat.id
    if 'nombre' not in chatbot_memory[chat_id]:  # Comprobamos si el usuario se registró
        bot.send_message(chat_id, "⚠️ Debes registrarte antes de usar este comando. Usa /Registro para registrarte. 📝")
        return

    bot.send_message(chat_id, "Por favor, escribe el nombre de la ciudad Española para obtener información.", reply_markup=types.ReplyKeyboardRemove())

    # Crear nuevo teclado con las ciudades españolas
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
    for ciudad in ciudades_espanolas:
        markup.add(types.KeyboardButton(ciudad))

    # Enviar botones de las ciudades
    bot.send_message(chat_id, "Puedes seleccionar una ciudad de la lista:", reply_markup=markup)

    bot.register_next_step_handler(message, fetch_city_info)


ciudades_espanolas = ['Madrid', 'Barcelona', 'Sevilla', 'Valencia', 'Bilbao', 'Malaga',
                      'Granada', 'Zaragoza', 'Toledo', 'Santander', 'Alicante',
                      'Cordoba', 'La Coruña', 'Valladolid', 'Salamanca', 'Pamplona',
                      'Logroño', 'Murcia', 'Oviedo', 'Palma de Mallorca', 'Málaga',
                      'Córdoba']


# Primero, Función para verificar si la ciudad está en España (utilizando una lista simple de ciudades)
def is_spanish_city(ciudad):   
    if ciudad.capitalize() in ciudades_espanolas:
        return True
    return False

# Creamos un diccionario para llevar el conteo de intentos por cada usuario
user_attempts = {}

# Función para buscar la información de la ciudad en GeoDB
def fetch_city_info(message):
    chat_id = message.chat.id
    ciudad = message.text.strip()

    # Validación de que la ciudad es española
    if not is_spanish_city(ciudad):
        bot.send_message(chat_id, "🌍 La ciudad debe ser española. Por favor, escribe el nombre de una ciudad española.")
        return

    # Construimos la URL con la ciudad y clave API
    url = f"{geodb_base_url}?namePrefix={ciudad}&countryIds=ES"

    headers = {
        "X-RapidAPI-Key": geodb_api_key,
        "X-RapidAPI-Host": "wft-geo-db.p.rapidapi.com"
    }

    # Realizamos la solicitud GET a la API
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()

        # Verificamos si se encontró información
        if data['data']:
            city = data['data'][0]            
            city_name = city.get('city', 'No disponible')
            country_name = city.get('country', 'No disponible')
            population = city.get('population', 'No disponible')
            latitude = city.get('latitude', 'No disponible')
            longitude = city.get('longitude', 'No disponible')
            region = city.get('region', 'No disponible')  # Región, si está disponible

            # Información formateada con emoticonos y negritas
            info_message = f"📍 **Información de {city_name}:**\n"
            info_message += f"🌍 **País:** {country_name}\n"
            info_message += f"🌆 **Población:** {population}\n"
            info_message += f"🌍 **Región:** {region}\n"
            info_message += f"📍 **Coordenadas:** _{latitude}_, _{longitude}_\n"
            # Enviar el mensaje con la información
            bot.send_message(chat_id, info_message, parse_mode="Markdown", reply_markup=types.ReplyKeyboardRemove())
        else:
            bot.send_message(chat_id, "❌ No se encontró información sobre esa ciudad. Por favor, intenta con otra.")
    else:
        bot.send_message(chat_id, "⚠️ Hubo un problema al obtener los datos de la ciudad. Intenta más tarde.")

    # Limpiar teclado anterior enviando un teclado vacío
    bot.send_message(chat_id, "Puedes seleccionar otra ciudad y consultarla si haces click en la lista", reply_markup=types.ReplyKeyboardRemove())

    # Crear nuevo teclado con las ciudades españolas
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
    for ciudad in ciudades_espanolas:
        markup.add(types.KeyboardButton(ciudad))

    # Enviar botones de las ciudades
    bot.send_message(chat_id, "Selecciona la lista si quieres:", reply_markup=markup)

    # Finalmente, enviamos los botones generales para que el usuario elija su siguiente acción
    final_message = (
        """🎉 Gracias por consultar la información de la ciudad. ¿Qué te gustaría hacer ahora? 🤔
        
        - /start o /help: Para iniciar otra vez
        - /Registro: Para cambiar tu nombre 📝
        - /Ciudad: Para seleccionar otra ciudad y mes y obtener análisis climático ☀️
        - /Mapa [Latitud, Longitud]: Ver el mapa de la ciudad deseada 🗺️"""
        )

    bot.send_message(chat_id, final_message)

#FUNCION CON API DE MAPBOX
@bot.message_handler(commands=['Mapa'])
def city_map(message):
    chat_id = message.chat.id
    user_input = message.text.strip()

    if not usuario: #COMO CON LOS OTROS DOS COMANDOS, PRIMERO QUEREMOS REGISTRO (AHORA USAMOS GLOBAL YA QUE EL MENSAJE LLEVA COORDENADAS)
        bot.send_message(chat_id, "⚠️ Debes registrarte antes de usar este comando. Usa /Registro para registrarte. 📝")
        return
    #Verificar si el usuario está registrado

    # Extraer las coordenadas después del comando
    if len(user_input.split()) > 1:
        try:
            # Procesar el input del usuario como coordenadas
            coords = " ".join(user_input.split()[1:]).replace(",", "").split()
            latitude = float(coords[0])
            longitude = float(coords[1])
        except (ValueError, IndexError):
            bot.send_message(chat_id, "Por favor, introduce coordenadas válidas en el formato: /Mapa <latitud> <longitud>. Ejemplo: /Mapa 40.253888888 -3.827777777")
            return
    else:
        bot.send_message(chat_id, "Por favor, introduce coordenadas en el formato: /Mapa <latitud> <longitud>. Ejemplo: /Mapa 40.253888888 -3.827777777")
        return

    # URL de Mapbox con la API Key
    api_key = "sk.eyJ1IjoieGFiaXp1bHVldGExNyIsImEiOiJjbTRpdmpra3gwOHF6MmlzYmlydTFzOGM1In0._ZVvcU8AdenfJI70Gtk79g"
    url = f"https://api.mapbox.com/styles/v1/mapbox/streets-v11/static/pin-l+000({longitude},{latitude})/{longitude},{latitude},12/600x400?access_token={api_key}"

    try:
        # Descargar la imagen del mapa
        response = requests.get(url)
        response.raise_for_status()  # Verifica si hubo un error HTTP
        # Enviar la imagen al usuario
        bot.send_photo(chat_id, photo=BytesIO(response.content), caption=f"🗺️ Mapa de las coordenadas ({latitude}, {longitude})")
    except requests.exceptions.RequestException as e:
        bot.send_message(chat_id, f"Error al generar el mapa: {e}")

    final_message = (
        """🎉 Gracias por consultar la información de la ciudad. ¿Qué te gustaría hacer ahora? 🤔
        
        - /start o /help: Para iniciar otra vez
        - /Registro: Para cambiar tu nombre 📝
        - /Ciudad: Para seleccionar otra ciudad y mes y obtener análisis climático ☀️
        - /Ciudad_info : Información geográfica de la ciudad que quieras mediante la API de GeoDB 🌐.
        - /Mapa [Latitud, Longitud]: Ver el mapa de otra ciudad deseada 🗺️"""
        )

    bot.send_message(chat_id, final_message)

# Iniciar el bot
bot.polling()